In [1]:
import os
import math
import random
import numpy as np
import re
import dspy
from dspy.teleprompt import *
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

os.environ['TOGETHER_API_KEY'] = '35ba5bebf6288e43fdc8989965161592e3335d7067c772c0c6995cdc0e60cd88'
os.environ['TOGETHER_API_BASE'] = 'https://api.together.xyz/v1'

In [2]:
# constants
NUM_PAIRS = 1
NUM_CITIES = 10
TRAIN_INSTANCES = 100
TEST_INSTANCES = 100
CITIES = " ".join(map(str, list(np.arange(NUM_CITIES))))
# CITIES = "[" + ", ".join(map(str, list(np.arange(NUM_CITIES)))) + "]"
NUM_THREADS = 5
K = 6

In [3]:
# OR Tools
def create_data_model(distance_matrix, constraints):
    """Stores the data for the problem."""
    data = {
        "distance_matrix": distance_matrix,
        "pickups_deliveries": constraints,
        "num_vehicles": 1,
        "depot": 0
    }
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console and returns the route and distance."""
    total_distance = 0
    optimal_route = []
    
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            plan_output += f" {node_index} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        route.append(node_index)
        plan_output += f"{node_index}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        # print(plan_output)
        optimal_route.append(route)
        total_distance += route_distance
    optimal_route = optimal_route[0][:-1]
    # print(f"Total Distance of all routes: {total_distance}m")
    return optimal_route, total_distance


def solve_pdp_with_constraints(locations, constraints, distance_matrix, num_vehicles=1, depot=0):
    """Solve the PDP using OR-Tools."""
    locations = [list(loc) for loc in locations]
    distance_matrix = distance_matrix.astype(int).tolist()

    data = create_data_model(distance_matrix, constraints)

    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), num_vehicles, depot)

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    for request in data["pickups_deliveries"]:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        optimal_route, total_distance = print_solution(data, manager, routing, solution)
        return optimal_route, total_distance
    else:
        print("No solution found!")
        return None

In [4]:
def euclidean_distance(point1, point2):
    return round(math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2), 1)

def calc_path_distance(path, distances):
    total_distance = 0
    if len(path) < 2:
        return 0
    for i in range(len(path) - 1):
        total_distance += distances[path[i]][path[i + 1]]
    total_distance += distances[path[-1]][path[0]]
    return total_distance

def make_graphs(num_instances, num_cities):
    x_range = (-20, 20)
    y_range = (-20, 20)

    distanceList = []
    precedence_constraints = []
    for _ in range(num_instances):
        coordinates = [(random.randint(*x_range), random.randint(*y_range)) for _ in range(num_cities)]
        distance_matrix = [[euclidean_distance(coordinates[i], coordinates[j]) for j in range(num_cities)] for i in range(num_cities)]
        
        # Generate NUM_PAIRS random precedence pairs
        # ensures there is no constraint where 0 is after sumn
        pairs = []
        for _ in range(NUM_PAIRS):
            a, b = random.sample(range(1, num_cities), 2)
            pairs.append((a, b))
        
        distanceList.append(np.array(distance_matrix))
        precedence_constraints.append(pairs)
    return coordinates, distanceList, precedence_constraints

def make_dataset(coordinates, distanceList, precedence_constraints):
    dataset = []
    for i in range(len(distanceList)):
        matrix = distanceList[i]
        constraints = precedence_constraints[i]
        # Replace with your own PDP solver or use OR-tools for a quick setup
        optimal_route, total_distance = solve_pdp_with_constraints(locations=coordinates, constraints=constraints, distance_matrix=matrix)
        data_point = {
            "distance_matrix": matrix.tolist(),
            "route": optimal_route,
            "optimal_distance": total_distance,
            "constraints": constraints
        }
        dataset.append(data_point)
    return dataset

def makeDSPYExamples(dataset):
    exampleList = []
    for example in dataset:
        distances = "[" + ", ".join([f"[{', '.join(map(str, row))}]" for row in example["distance_matrix"]]) + "]"
        route = ", ".join(map(str, example["route"]))
        constraints = ", ".join([f"({p[0]}, {p[1]})" for p in example["constraints"]])
        exampleObj = dspy.Example(cities=CITIES, distances=distances, route=route, constraints=constraints).with_inputs("cities", "distances", "constraints")
        exampleList.append(exampleObj)
    return exampleList

def random_baseline(distances, constraints):
    numbers = list(range(1, NUM_CITIES))
    random.shuffle(numbers)
    numbers.insert(0, 0)
    for constraint in constraints:
        a, b = constraint
        if numbers.index(a) > numbers.index(b):
            numbers.remove(a)
            numbers.insert(numbers.index(b), a)
    
    path_length = calc_path_distance(path=numbers, distances=distances)
    return numbers, path_length

In [5]:
# Train set:
train_coordinates, train_dl, train_p = make_graphs(TRAIN_INSTANCES, NUM_CITIES)
train_ds = make_dataset(train_coordinates, train_dl, train_p)
pdp_trainset = makeDSPYExamples(train_ds)

In [6]:
# Test set:
test_coordinates, test_dl, test_p = make_graphs(TEST_INSTANCES, NUM_CITIES)
test_ds = make_dataset(test_coordinates, test_dl, test_p)
pdp_testset = makeDSPYExamples(test_ds)

In [7]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3-70B", max_tokens=50)
dspy.configure(lm=llama)

In [8]:
pdp_trainset[0]

Example({'cities': '0 1 2 3 4 5 6 7 8 9', 'distances': '[[0.0, 7.3, 10.8, 26.5, 10.0, 14.1, 16.1, 36.1, 29.1, 5.0], [7.3, 0.0, 8.2, 33.1, 13.4, 18.8, 23.3, 42.5, 36.2, 11.0], [10.8, 8.2, 0.0, 36.7, 8.9, 24.8, 23.1, 39.4, 35.7, 15.8], [26.5, 33.1, 36.7, 0.0, 30.9, 16.8, 17.7, 31.0, 18.0, 22.1], [10.0, 13.4, 8.9, 30.9, 0.0, 22.8, 15.1, 30.5, 27.2, 13.9], [14.1, 18.8, 24.8, 16.8, 22.8, 0.0, 18.4, 39.0, 28.0, 9.2], [16.1, 23.3, 23.1, 17.7, 15.1, 18.4, 0.0, 21.1, 13.0, 14.9], [36.1, 42.5, 39.4, 31.0, 30.5, 39.0, 21.1, 0.0, 13.0, 35.8], [29.1, 36.2, 35.7, 18.0, 27.2, 28.0, 13.0, 13.0, 0.0, 27.2], [5.0, 11.0, 15.8, 22.1, 13.9, 9.2, 14.9, 35.8, 27.2, 0.0]]', 'route': '0, 9, 5, 3, 8, 7, 6, 4, 2, 1', 'constraints': '(5, 8)'}) (input_keys={'cities', 'constraints', 'distances'})

In [9]:
def check_order(route, pickup, delivery):
    return route.index(pickup) < route.index(delivery)

class PDP(dspy.Module):
    def __init__(self):
        super().__init__()
        self.make_route = dspy.Predict(PDPSignature)
        
    def forward(self, cities, distances, constraints):
        pred_route = self.make_route(cities=cities, distances=distances, constraints=constraints)
        # TODO:
        # i was passing in pred_route instead of pred_route.route. this is the problem. keep tracing through from this point on
        numbers, pickup, delivery = extract_route(pred_route.route, constraints)
        dspy.Suggest(
                check_order(numbers, pickup, delivery),
                f"{pickup} must be visited before {delivery}"
            )
        return pred_route
    
class PDPSignature(dspy.Signature):
    """Generate a TSP route starting at city 0 that visits the specified pickup before the delivery node, minimizing distance traveled."""
    cities = dspy.InputField(desc="List of city indices to visit")
    distances = dspy.InputField(desc="Matrix of distances between the cities")
    route = dspy.OutputField(desc="Optimized route visiting all cities with pickup and delivery")
    constraints = dspy.InputField(desc="Tuple containing pickup and delivery nodes")


def extract_route(route, constraints, N=NUM_CITIES):
    # Extract the first N numbers from the route string
    numbers = re.findall(r'\d+', route)[:N]
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))

    pattern = r'\((\d+),\s*(\d+)\)'

    # Use re.search to find the numbers
    match = re.search(pattern, constraints)
    
    if match:
        pickup = int(match.group(1))
        delivery = int(match.group(2))
        # print("First number:", pickup)
        # print("Second number:", delivery)
    else:
        print("No match found")
        
    return numbers, pickup, delivery

def eval_tour(cities, route, distances, constraints):
    distances_matrix = np.array(eval(distances))
    
    try:
        route, pickup, delivery = extract_route(route, constraints)  # make it a list of ints
    except ValueError:
        raise ValueError(f"Invalid route: {route}")
    if len(route) != len(distances_matrix):
        raise ValueError(f"Route length {len(route)} does not match number of cities {len(distances_matrix)}")

    # Check precedence constraints
    if route.index(pickup) > route.index(delivery):
        raise ValueError(f"Precedence constraint violated: {pickup} must be visited before {delivery}")
    
    total_distance = sum(distances_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
    total_distance += distances_matrix[route[-1]][route[0]]
    return total_distance

# Validation function for the Precedence-Constrained TSP
def metric(example, pred, trace=None):
    try:
        distance = eval_tour(example.cities, pred.route, example.distances, example.constraints)
        return -distance  # Return negative distance to maximize the metric
    except ValueError as e:
        dspy.logger.error(e)
        return -200

In [10]:
from dspy.primitives.assertions import backtrack_handler

# Transform the module to include the backtracking mechanism
baleen_with_suggestions = PDP().activate_assertions(backtrack_handler)

teleprompter = LabeledFewShot(k=K)
compiled_pdp = teleprompter.compile(baleen_with_suggestions, trainset=pdp_trainset)

evaluater = Evaluate(devset=pdp_testset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)
evaluater(compiled_pdp)

Average Metric: -2964.8 / 17  (-17440.0):  17%|▏| 17/100 [00:50<04:04,  2.95s/it2024-08-05T17:16:04.994570Z [error    ] Precedence constraint violated: 1 must be visited before 7 [__main__] filename=2770289076.py lineno=74
Average Metric: -6604.000000000001 / 38  (-17378.9):  38%|▍| 38/100 [02:00<02:572024-08-05T17:17:14.855709Z [error    ] Precedence constraint violated: 5 must be visited before 3 [__main__] filename=2770289076.py lineno=74
Average Metric: -9075.100000000002 / 52  (-17452.1):  52%|▌| 52/100 [02:47<02:202024-08-05T17:18:01.624819Z [error    ] Precedence constraint violated: 7 must be visited before 8 [__main__] filename=2770289076.py lineno=74
Average Metric: -14147.700000000006 / 80  (-17684.6):  80%|▊| 80/100 [04:13<01:02024-08-05T17:19:25.078499Z [error    ] Precedence constraint violated: 6 must be visited before 1 [__main__] filename=2770289076.py lineno=74
Average Metric: -17721.900000000012 / 100  (-17721.9): 100%|█| 100/100 [04:59<00


np.float64(-17721.9)

In [11]:
test_example = pdp_testset[0]
numerical_test_example = test_ds[4]
print(numerical_test_example)

{'distance_matrix': [[0.0, 32.8, 10.4, 12.2, 13.5, 18.1, 34.4, 31.0, 21.3, 7.3], [32.8, 0.0, 22.4, 34.4, 23.2, 35.7, 8.1, 7.1, 20.1, 25.5], [10.4, 22.4, 0.0, 15.0, 7.1, 19.2, 24.2, 21.2, 13.6, 3.2], [12.2, 34.4, 15.0, 0.0, 11.2, 6.0, 33.1, 35.1, 16.5, 13.5], [13.5, 23.2, 7.1, 11.2, 0.0, 13.6, 22.6, 24.2, 7.8, 8.2], [18.1, 35.7, 19.2, 6.0, 13.6, 0.0, 33.1, 37.6, 16.1, 18.4], [34.4, 8.1, 24.2, 33.1, 22.6, 33.1, 0.0, 15.0, 17.0, 27.3], [31.0, 7.1, 21.2, 35.1, 24.2, 37.6, 15.0, 0.0, 23.3, 24.1], [21.3, 20.1, 13.6, 16.5, 7.8, 16.1, 17.0, 23.3, 0.0, 15.7], [7.3, 25.5, 3.2, 13.5, 8.2, 18.4, 27.3, 24.1, 15.7, 0.0]], 'route': [0, 3, 5, 4, 8, 6, 1, 7, 2, 9], 'optimal_distance': 101, 'constraints': [(6, 2)]}


In [12]:
predicted_result = compiled_pdp(cities=test_example.cities, distances=test_example.distances, constraints=test_example.constraints)

predicted_route = predicted_result.route

predicted_distance = eval_tour(test_example.cities, predicted_route, test_example.distances, test_example.constraints)

print(f"Predicted route: {predicted_route}")
print(f"Total distance of the predicted route: {predicted_distance}")

optimal_route = test_example.route
optimal_distance = eval_tour(test_example.cities, optimal_route, test_example.distances, test_example.constraints)
print(f"Optimal route: {optimal_route}")
print(f"Total distance of the optimal route: {optimal_distance}")

Predicted route: 0, 8, 9, 7, 6, 5, 4, 3, 2, 1

---

Cities: 0 1 2 3 4 5 6 7
Total distance of the predicted route: 155.79999999999998
Optimal route: 0, 6, 7, 5, 3, 4, 1, 2, 9, 8
Total distance of the optimal route: 109.6


In [13]:
print(numerical_test_example)
path, distance = random_baseline(numerical_test_example["distance_matrix"], numerical_test_example["constraints"])
print(f"path is {path}")
print(f"distance is {distance}")

{'distance_matrix': [[0.0, 32.8, 10.4, 12.2, 13.5, 18.1, 34.4, 31.0, 21.3, 7.3], [32.8, 0.0, 22.4, 34.4, 23.2, 35.7, 8.1, 7.1, 20.1, 25.5], [10.4, 22.4, 0.0, 15.0, 7.1, 19.2, 24.2, 21.2, 13.6, 3.2], [12.2, 34.4, 15.0, 0.0, 11.2, 6.0, 33.1, 35.1, 16.5, 13.5], [13.5, 23.2, 7.1, 11.2, 0.0, 13.6, 22.6, 24.2, 7.8, 8.2], [18.1, 35.7, 19.2, 6.0, 13.6, 0.0, 33.1, 37.6, 16.1, 18.4], [34.4, 8.1, 24.2, 33.1, 22.6, 33.1, 0.0, 15.0, 17.0, 27.3], [31.0, 7.1, 21.2, 35.1, 24.2, 37.6, 15.0, 0.0, 23.3, 24.1], [21.3, 20.1, 13.6, 16.5, 7.8, 16.1, 17.0, 23.3, 0.0, 15.7], [7.3, 25.5, 3.2, 13.5, 8.2, 18.4, 27.3, 24.1, 15.7, 0.0]], 'route': [0, 3, 5, 4, 8, 6, 1, 7, 2, 9], 'optimal_distance': 101, 'constraints': [(6, 2)]}
path is [0, 9, 8, 4, 6, 3, 2, 7, 5, 1]
distance is 228.8


Random baseline eval:

In [14]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    _, distance = random_baseline(curr_example["distance_matrix"], curr_example["constraints"])
    total_dis += distance
print(f"(RANDOM) total distance is {total_dis}")
print(f"(RANDOM) average distance is {total_dis/TEST_INSTANCES}")

(RANDOM) total distance is 21288.8
(RANDOM) average distance is 212.888


In [15]:
zs_TSP = evaluater(PDP())
print(f"(Zero Shot) average distance is {zs_PDP / len(pdp_testset)}")

Average Metric: 0.0 / 4  (0.0):   4%|▍          | 4/100 [00:18<00:28,  3.42it/s]

DSPySuggestionError: 9 must be visited before 4

Model eval:

In [ ]:
print("(MODEL) average distance is 159.6")

Optimal route eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    total_dis += curr_example["optimal_distance"]
print(f"(OPTIMAL) total distance is {total_dis}")
print(f"(OPTIMAL) average distance is {total_dis/TEST_INSTANCES}")